In [24]:
from Bio.Align import MultipleSeqAlignment
from Bio import SeqIO
from Bio.pairwise2 import format_alignment
from Bio import pairwise2
from Bio import Entrez
import pandas as pd  
import numpy as np
from Bio.Seq import Seq
# import matplotlib.pyplot as plt
# import biotite
# import biotite.sequence as seq
# import biotite.sequence.align as align
# import biotite.sequence.io.fasta as fasta
# import biotite.database.entrez as entrez
# import biotite.sequence.graphics as graphics
# from biotite.sequence import Alphabet
# from biotite.sequence.align import SubstitutionMatrix
# from dna_features_viewer import GraphicFeature, GraphicRecord
# import matplotlib.pyplot as plt

In [13]:
# train_genomes_list = ['NC_019910.1', #Salmonella phage SKML-39, Caudovirales; Ackermannviridae; Aglimvirinae; Ag3virus
#                           'JN593240.1',  #Escherichia virus CBA120, Caudovirales; Ackermannviridae; Cvivirinae; Cba120virus
#                            'NC_019507.1', #Campylobacter phage CP21, Caudovirales; Myoviridae; Cp220virus
#                            'NC_021337.1', #Acinetobacter phage AB3, Caudovirales; Podoviridae; Autographivirinae
#                            'NC_028904.1', #Streptomyces virus Amela, Caudovirales; Siphoviridae; Arquatrovirinae; Camvirus
#                            'NC_023697.1', #Mycobacterium phage Babsiella, Caudovirales; Siphoviridae; Chebruvirinae; Brujitavirus
#                            'NC_018273.1', #Leuconostoc phage Lmd1, Caudovirales; Siphoviridae; Mccleskeyvirinae; Lmd1virus
#                            'NC_029099.1', #Klebsiella phage KP36, Caudovirales; Siphoviridae; Tunavirinae; Kp36virus
#                            'NC_025434.1', #Shigella phage POCJ13, Caudovirales; Podoviridae; Sepvirinae; Pocjvirus
#                            'NC_005083.2', #Vibrio virus KVP40, Caudovirales; Myoviridae; Tevenvirinae; Schizot4virus
#                          ]

### Input: list of accesion numbers for the database 

In [ ]:
file_with_acc_numbers = 'test.acc_lst'

In [ ]:
genomes_list = [x.strip() for x in open(file_with_acc_numbers, 'r').readlines()]

###  Retrieving the sequence and the features by the accesion number

In [2]:
def retrieve_seq (acc_num): 
    Entrez.email = "E.Deyneka@student.tudelft.nl"
    handle = Entrez.efetch(db="nuccore",
                       id=acc_num,
                       rettype="gb",
                       retmode="text")
    entry = SeqIO.read(handle, "genbank")
    return entry

### Safe genome in fasta format

In [24]:
def save_fasta (genome, name):
    f = open(name, "w")
    f.write(">%s\n" %genome.name)
    f.write(str(genome.seq))
    f.write('\n')

In [25]:
for item in genomes_list:
    genome = retrieve_seq (item)
    save_fasta (genome, "{}.fasta".format(genome.name))

### After saving the sequences in .fasta format, we need to generate reads. The output of deepsim - to /scratch/ only reads and mapping to home directory

.fastq and .paf file should have the same name as genome accession number

### Parsing DeepSim reads 

In [3]:
# input - .fastq reads, output - dictionary with DeepSim identifier and my identifier and read itself

# filename - 'acc_num.fastq'
def open_reads (filename, filtering):
    my_identifiers = [] #reads and my identifiers
    ids_deepsim = []
    reads = []
    for i, record in enumerate(SeqIO.parse(filename, "fastq")):
        if filtering != None:
            if len(str(record.seq)) >= filtering:
                my_identifiers.append('{}-{}'.format(filename[:-6], (i+1)))
                reads.append(str(record.seq))
                ids_deepsim.append(record.id)     

        
    id_dictionary = dict(zip(ids_deepsim, my_identifiers))
    reads_dictionary = dict(zip(my_identifiers, reads))
    return id_dictionary, reads_dictionary 



#### Note that now some reads can be missing because of filtering 

In [5]:
# Save feads in the format for BLAST, input - reads dictionary
def save_reads(reads_dictionary):
    f = open("Reads_{}.fna".format(list(reads_dictionary.keys())[0].split('-')[0]), "w")
    for i in range(len(reads_dictionary)):
        f.write(">{}\n".format(list(reads_dictionary.keys())[i]))
        f.write(list(reads_dictionary.values())[i])
        f.write('\n')

In [46]:
def mapping(filename):
    df = pd.read_table(filename, header=None)
    d = {'Read number': [], 'Strand': [], 'Location of gene in the read': []}
    ground_truth = pd.DataFrame(data=d)

    for i in range(df.shape[0]):
        if id_dictionary.get(df.iloc[i,0]) != None:
            start_for = df.iloc[i,7]
            end_for = df.iloc[i,8]

            start_rev = df.iloc[i,6]-df.iloc[i,8]
            end_rev = df.iloc[i,6] - (df.iloc[i,7]-1)
            subject_acc = df.iloc[i,5] 


            #retrieving subject genome
            subject_genome = retrieve_seq (subject_acc)

            #location of mapped regions
            idx_ref_for = pd.Index(np.arange(start_for, end_for))
            idx_ref_rev = pd.Index(np.arange(start_rev, end_rev))
            for j in range(len(subject_genome.features)):
                if subject_genome.features[j].type=='CDS':
                    # for the forward strand
                    ground_truth = gene_locations(df.iloc[i,0], ground_truth, subject_genome.features[j], idx_ref_for, 1, start_for, end_for)

                    # for the reverse strand
                    ground_truth = gene_locations(df.iloc[i,0], ground_truth, subject_genome.features[j], idx_ref_rev, -1, start_rev, end_rev)
    return ground_truth

In [47]:
def gene_locations(read_id, ground_truth, subject_genome_feature, idx_ref, strand, start, end): # subject_genome_features[j]
    idx = pd.Index(np.arange(subject_genome_feature.location.start.position, 
                             subject_genome_feature.location.end.position))
    overlap = list(idx_ref.intersection(idx))
    if len(overlap) != 0:
        overlap_start = overlap[0]
        overlap_end = overlap[-1]
        ground_truth = ground_truth.append({'Read number': id_dictionary.get(read_id), 'Strand': strand, 
                                'Location of gene in the read': [overlap_start - start, 
                                                                 overlap_end - start+1 ]} , ignore_index=True) # 0-based system
    return ground_truth

### Main code     
 

In [10]:
id_dictionary, reads_dictionary = open_reads ('NC_019910.fastq', 2000)


In [48]:
ground_truth = mapping('NC_019910.paf')

C:\Users\Kate\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [62]:
def writing_input(strand, df_one_read):
    f.write(df_one_read.loc[0, 'Read number'] + ';')
    seq = reads_dictionary[df_one_read.loc[0, 'Read number']] # retrieve seq from dictionary 
    if strand == 1:
        f.write(seq + ';')
    else:
        f.write(str(Seq(seq).reverse_complement()) + ';')
        
    locations = []
    encoded_seq = np.zeros(len(seq), dtype=int)
    for i in range(df_one_read.shape[0]):
        if df_one_read.loc[i, 'Strand'] == strand:
            locations.append(df_one_read.loc[i, 'Location of gene in the read'])
            start = df_one_read.loc[i, 'Location of gene in the read'][0]
            stop = df_one_read.loc[i, 'Location of gene in the read'][1]
            encoded_seq[start:stop] = 1
            
    f.write(str(locations) + ';')
    f.write(''.join(str(x) for x in encoded_seq) + ';')
    if strand == 1:
        f.write('forward' + '\n')
    else: 
        f.write('reverse' + '\n')

#### This code creates input that we discussed  

In [63]:
f = open('Input_file_reads.txt', "w")
_, idx = np.unique(ground_truth[['Read number']].values, return_index=True)
for read_num in [x[0] for x in ground_truth[['Read number']].values[np.sort(idx)]]:

    df_one_read = ground_truth.loc[ground_truth['Read number'] == read_num].reset_index()
    writing_input(1, df_one_read)
    writing_input(-1, df_one_read)
